# ASSET demonstration

This notebook aims at showing how to use the ASSET package to extract a spectrum
from log-slit spectroscopy data. This example extracts the spectrum of a star
companion from coronagraphic data.


In [1]:
# Load packages
using DelimitedFiles
using InterpolationKernels
using LinearInterpolators
using InverseProblem

using ASSET

## Generation of a Toy Model

The following cells generate a Toy Model to present how to use ASSET.
The spectrum used in the Toy Model was obtained from the P330 E CALSPEC reference spectrum (Bohlin, Gordon & Tremblay 2014). We use a version of this spectrum sampled with 1024 points between 0.5 and 10 microns.
The PSF used in the Toy Model was obtained from the P330 E MIRI LRS data (see Figure 3. and 4. of the paper). We used the two-order non-parametric PSF.
We generate some spatial and spectral coordinate maps align on the grid.

In [ ]:
# Interpolation kernel 
ker = CatmullRomSpline(Float64, Flat)

# Get the Toy Model spectrum
lz = readdlm("p330e_for_toymodel.txt");
l_tm=lz[:,1]
z_tm=lz[:,2]
λmin=l_tm[1]
λmax=l_tm[end]

Nz=length(z_tm)
# Get the Toy Model PSF and its data
h2=readdlm("nonparametric_parameters_at_order_2.txt") #FIXME : the PSF is not centered in psf_size/2, may be problematic? (It is here but just for data generation)
shift2=0.58 # generate an arbitrary shift of the PSF
order2=size(h2)[2]
psf_size=size(h2)[1]

# Generate an arbitrary regularization for the PSF structure
hph=1e0
wh=cat(ones(psf_size,order2),zeros(psf_size,order2),dims=3)
Rh=WeightedTikhonov(hph, wh);

# Generate a non-parametric PSF of order 2
h_tm=ASSET.SeriesExpansionPSF(h2, shift2,ker ,Rh)

SeriesExpansionPSF{Float64, Matrix{Float64}, CatmullRomSpline{Float64, Flat}, WeightedTikhonov{Float64}}([1.2206859465191305e-178 2.3931578518560086e-178; 2.6672069254846737e-173 5.22906515244003e-173; … ; 1.2036534537864966e-169 2.325974592264908e-169; 1.2029757830162813e-174 2.3246650439881643e-174], 0.58, CatmullRomSpline{Float64, Flat}(), Weighted Tikhonov:
 - level `mu` : 1.0
 - weights` : [1.0 1.0; 1.0 1.0; … ; 1.0 1.0; 1.0 1.0;;; 0.0 0.0; 0.0 0.0; … ; 0.0 0.0; 0.0 0.0])

In [3]:
m = 840 # Number of columns in the data
n = 42  # Number of rows in the data
lambda = repeat(collect(range(λmin, stop=λmax, length=m))', outer=(n,1,2)) # generate a spectral map for two dithers
rho = cat(repeat(collect(range(-n/3, step=1, length=n)), outer=(1,m)),
          repeat(collect(range(-2*n/3, step=1, length=n)), outer=(1,m)), dims=3) #generate spatial map for two dithers


λz=range(λmin, length=Nz, stop=λmax); #Sampling for the spectrum extraction
F = SparseInterpolator(ker, lambda, λz) #Interpolator for the spectrum

H=zeros(n,m,2)
psf_map!(H, h_tm, rho, lambda)
tm=H.*(F*z_tm)

data = tm .+ sqrt.(tm .+ 1e-6).*randn(n,m,2) #Generate some data with i.i.d photon noise and some readout noise of var 1e-6
wgt = (rand(n,m,1) .< 0.9)./(tm .+ 1e-6) #Generate some weiths as the inverse of the diagonal covariance of the data and with some defective pixels

D = CalibratedData(data, wgt, rho, lambda)

CalibratedData{Float64}:
 - scientific data `d` : Array{Float64, 3}
 - weight of data `w` : Array{Float64, 3}
 - spatial map `ρ_map` : Array{Float64, 3}
 - spectral map `λ_map` : Array{Float64, 3}

## Extractions examples


In [4]:
hpz=1e-4 #regularization parameter for the spectrum
hpb=1e1  #regularization parameter for the background
Rz=WeightedTikhonov(hpz, ones(Nz,1)) #Spectrum regularization
Rb=hpb*tikhonov() #Background regularization

b=ASSET.BkgMdl(zeros(n,m), Rb) #Background model

BkgMdl{Float64, 2, @NamedTuple{lower::Float64}}([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], HomogenRegul:
 - level `mu` : 10.0
 - function `func` : InverseProblem.Tikhonov()
 - use direct inversion `direct_inversion` : true
 - degree `deg` : 2.0, (lower = 0.0,))

###  Parametric models
#### Chromatic Gaussian

In [5]:
parinit = maximum(lambda)^(-2)
psf_params_bnds=[(1e-8, 1.)] #The bounds of the gaussian psf parameter for the autocalibration
center_bnds=2. #The bounds of the gaussian center for the autocalibration
h_pg=ASSET.chromGaussianPSF(parinit)
z_pg=zeros(Nz); 

z_fpg,h_fpg =ASSET.extract_spectrum!(z_pg, F, h_pg, D, Rz, b;
                                    auto_calib=Val(true),
                                    max_iter=10,
                                    extract_kwds=(psf_params_bnds=psf_params_bnds,
                                                  psf_center_bnd=center_bnds,
                                                  max_iter=10))

LoadError: StackOverflowError:

#### Chromatic Moffat

In [ ]:
parinit = [hfpg.a, 2.]
psf_params_bnds=[(0., 3.);(1., 3.)]
center_bnds=2.
h_pm=ASSET.chromMoffatPSF(parinit)
z_pm=zeros(Nz)

z_fpm,h_fpm =ASSET.extract_spectrum!(z_pm, F, h_pm, D ,Rz, b;
                                    auto_calib=Val(true),
                                    max_iter=100,
                                    extract_kwds=(psf_params_bnds=psf_params_bnds, 
                                                    psf_center_bnd=center_bnds,
                                                    max_iter=10))        

### Series expansion PSF

#### With one order

In [ ]:
order=1

shift_bnds=(-1.,1.)
center_bnds=2.
hph=1e0 #regularization parameter of the PSF
wh=cat(ones(psf_size,order),zeros(psf_size,order),dims=3) #weights of the PSF
Rh=WeightedTikhonov(hph, wh);

h_se1=ASSET.SeriesExpansionPSF(zeros(psf_size,order), 0.0,ker ,Rh)
z_se1=copy(z_pm) #initiliaze spectrum with an approximation of the solution to avoid local minima

z_fse1,h_fse1 =ASSET.extract_spectrum!(z_se1, F, h_se1, D ,Rz, b;
            auto_calib=Val(true),
            max_iter=100,
            extract_kwds=(psf_center_bnd=center_bnds,
                          psf_shift_bnds=shift_bnds,
                          max_iter=10))

#### With two orders

In [ ]:
order=2

shift_bnds=(-1.,1.)
center_bnds=2.
hph=1e0 #regularization parameter of the PSF
wh=cat(ones(psf_size,order),zeros(psf_size,order),dims=3)
Rh=WeightedTikhonov(hph, wh);

h_se2=ASSET.SeriesExpansionPSF(zeros(psf_size,order), 0.0, ker ,Rh)
z_se2=copy(z_pm) #initiliaze spectrum with an approximation of the solution to avoid local minima

z_fse2,h_fse2=ASSET.extract_spectrum!(z_se2, F, h_se2, D ,Rz, b;
            auto_calib=Val(true),
            max_iter=100,
            extract_kwds=(psf_center_bnd=center_bnds,
                          psf_shift_bnds=shift_bnds,
                          max_iter=10))                         